In [ ]:
import json
import pandas as pd
from openai import OpenAI
from io import StringIO
import re


In [ ]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [ ]:
client = OpenAI(api_key=k)

In [ ]:
#!pip install openai -U

In [ ]:
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel
from pydantic import Field
from openai import OpenAI

client = OpenAI(api_key=k)

class Category(Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

class Brand(BaseModel):
    name: str
    characteristics: List[str] = Field(None, description="Fabrics the brand uses.")

class Brand_answer(BaseModel):
    a: List[Brand]


completion_song = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "Name 10 best selling songs and for each song list 2 single word dominant emotions that the song inspires. No singer names."}
    ],
    response_format=Brand_answer,
)

In [ ]:
client = OpenAI(api_key=k)

class Category(Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

class Brand(BaseModel):
    name: str
    characteristics: List[str] = Field(None, description="Fabrics the brand uses.")

class Brand_answer(BaseModel):
    a: List[Brand]


completion_brand = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "Give me a list of 10 top selling both gender clothing brand names in US."}
    ],
    response_format=Brand_answer,
)

In [ ]:
[i.values() for i in list(songs.values())[0]]

In [ ]:
from enum import Enum

class Characteristic(Enum):
    SAD = "sad"
    JOYFUL = "joyful"
    ANGRY = "angry"
    CALM = "calm"
    ENERGETIC = "energetic"
    PEACEFUL = "peaceful"
    FUNKY = "funky"
    VIBRANT = "vibrant"
    MELLOW = "mellow"
    MELANCHOLIC = "melancholic"
    EXUBERANT = "exuberant"
    SERENE = "serene"
    MYSTERIOUS = "mysterious"
    SOMBER = "somber"
    CHEERFUL = "cheerful"
    DRAMATIC = "dramatic"
    PLAYFUL = "playful"
    ROMANTIC = "romantic"
    DREAMY = "dreamy"
    BOLD = "bold"
    DELICATE = "delicate"
    WHIMSICAL = "whimsical"
    ELEGANT = "elegant"
    GRITTY = "gritty"
    GLOOMY = "gloomy"
    ETHEREAL = "ethereal"
    SPIRITED = "spirited"
    LYRICAL = "lyrical"
    TENSE = "tense"
    SULTRY = "sultry"
    INTENSE = "intense"
    TRANQUIL = "tranquil"
    EXCITING = "exciting"
    WARM = "warm"
    COOL = "cool"
    HOPEFUL = "hopeful"
    ANXIOUS = "anxious"
    INSPIRING = "inspiring"
    DYNAMIC = "dynamic"
    EERIE = "eerie"
    SENSUAL = "sensual"
    REBELLIOUS = "rebellious"
    NOSTALGIC = "nostalgic"
    UPLIFTING = "uplifting"
    REFLECTIVE = "reflective"
    PASSIONATE = "passionate"
    SOPHISTICATED = "sophisticated"
    SURREAL = "surreal"
    JUBILANT = "jubilant"
    BITTERSWEET = "bittersweet"
    GRACEFUL = "graceful"
    HEROIC = "heroic"
    RUSTIC = "rustic"
    URBAN = "urban"
    MAJESTIC = "majestic"
    FESTIVE = "festive"
    MINIMALIST = "minimalist"
    BAROQUE = "baroque"
    MONOCHROME = "monochrome"

class SongAttributes(BaseModel):
    song_name: str
    characteristics: List[Characteristic]


In [ ]:
class SongAttributes(BaseModel):
    song_name: str
    characteristics: List[str]

In [ ]:
#you can also procees from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = completion_song.choices[0].message.content
songs = json.loads(out)
songs
df = pd.DataFrame(data = [i.values() for i in list(songs.values())[0]], columns = ['name','characteristic'])
df.set_index('name', inplace=True)
df.to_dict().values()

In [ ]:
from transformers import BertTokenizer, BertModel
#import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


In [ ]:
import numpy as np

def average_embedding(embeddings):
    return np.mean(embeddings, axis=0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = []
for adj_A in list(d_b.values()):
    avg_embedding_A = average_embedding(get_embeddings(adj_A))
    for adj_B in list(d_s.values()):
        avg_embedding_B = average_embedding(get_embeddings(adj_B))
        similarity_score = cosine_similarity([avg_embedding_A], [avg_embedding_B])[0][0]
        similarities.append((adj_A, adj_B, similarity_score))